<a href="https://colab.research.google.com/github/jjangmo91/ParkLab/blob/main/project/2025%20NASA%20Space%20Apps%20Challenge/01_mekong_flood_analysis_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터셋
* Landsat5 TM C2 L2 / USGS, NASA / SR_B2(Green), SR_B5(SWIR) / 2005-2008 / LANDSAT/LT05/C02/T1_L2
* Sentinel-1 / ESA / VV, VH / 2015-2024 / COPERNICUS/S1_GRD
* JRC Global Surface Water / JRC, Google / seasonality, transition, etc. / 1984-2021 / JRC/GSW1_4/GlobalSurfaceWater
* CHIRPS Daily / UCSB, CHG /precipitation / 1981-현재 / UCSB-CHG/CHIRPS/DAILY

In [ ]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()
# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91') # 본인 계정

# Phase 1: 신뢰도 높은 기준선 설정(Landsat & JRC 데이터를 이용한 교차 검증)

* 목표: 댐 영향이 본격화 되기 이전 메콩강 하류 모습 정의
* step1: 4년간(2005-2008)의 Landsat 데이터로 평균적인 건기/우기 기준선을 생성
* step2: Landsat 기반 우기 수역과 JRC의 영구 수역을 비교하여 기준선을 검증
* 결과: 댐 영향 이전의 기준선 설정

In [ ]:
# 분석 파라미터 설정
# Area of Interset, AOI: 캄보디아 프놈펜, 톤레삽 호수, 베트남 메콩 델타 포함
aoi = ee.Geometry.Rectangle([103.0, 9.5, 107.0, 13.5])

# 기준선 분석 기간 (댐 영향 이전 / 4년치 확장)
pre_dam_start_date = '2005-01-01'
pre_dam_end_date = '2008-12-31'

In [ ]:
# Landsat Collection 2 스케일링 및 구름 마스킹 함수 정의
def scale_and_mask_l5c2(image):
    # QA_PIXEL 밴드에서 구름(비트3)과 구름 그림자(비트4) 정보 추출
    cloud_shadow_bit_mask = 1 << 4
    clouds_bit_mask = 1 << 3
    qa_pixel = image.select('QA_PIXEL')

    # 구름과 그림자가 없는 깨끗한 픽셀만 선택
    mask = qa_pixel.bitwiseAnd(cloud_shadow_bit_mask).eq(0) \
                   .And(qa_pixel.bitwiseAnd(clouds_bit_mask).eq(0))

    # 광학 밴드(SR_B*)에 스케일 팩터와 오프셋 적용
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    # 열 밴드(ST_B6)는 다른 스케일을 가지므로 따로 처리
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)

    # 스케일링된 밴드와 마스크를 원본 이미지에 다시 추가하고 적용
    return image.addBands(optical_bands, None, True) \
                .addBands(thermal_bands, None, True) \
                .updateMask(mask)

# 4년치 평균적인 건기/우기 자연색 이미지 생성
landsat_collection = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate(pre_dam_start_date, pre_dam_end_date) \
    .map(scale_and_mask_l5c2)

# 건기(Dry Season 11월~4월) 데이터 필터링 후 중간값(median)으로 합성
landsat_dry_baseline = landsat_collection.filter(ee.Filter.calendarRange(11, 4, 'month')).median().clip(aoi)

# 우기(Wet Season 5~10월) 데이터 필터링 후 중간값(median)으로 합성
landsat_wet_baseline = landsat_collection.filter(ee.Filter.calendarRange(5, 10, 'month')).median().clip(aoi)

# Landsat 기반 우기 '수역 마스크' 생성 및 면적 계산
def calculate_mndwi(image):
    return image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

mndwi_wet_baseline = calculate_mndwi(landsat_wet_baseline)
water_mask_baseline = mndwi_wet_baseline.gt(0).selfMask()

baseline_area = water_mask_baseline.multiply(ee.Image.pixelArea()) \
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e12
    )
baseline_wet_area_km2 = ee.Number(baseline_area.get('MNDWI')).divide(1e6).getInfo()
print(f"계산된 과거 기준선(2005-2008) 평균 우기 면적: {baseline_wet_area_km2:,.2f} km²")

# JRC 데이터를 이용한 교차 검증
jrc_data = ee.Image('JRC/GSW1_4/GlobalSurfaceWater').clip(aoi)

# 'seasonality' 밴드에서 값이 1인 '영구 수역'만 추출
jrc_permanent_water = jrc_data.select('seasonality').eq(1).selfMask()

# 시각화 파라미터 (Collection 2에 맞게 밴드 이름과 값 범위 수정)
vis_params_landsat = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'], 'min': 0.0, 'max': 0.3} # SR값 범위

# 지도 생성 (하나의 지도로 통합)
m_baseline = geemap.Map(center=[11.5, 105.5], zoom=8)

# 지도에 레이어 추가
m_baseline.addLayer(landsat_dry_baseline, vis_params_landsat, '기준선: 건기 모습 (Landsat)')
m_baseline.addLayer(landsat_wet_baseline, vis_params_landsat, '기준선: 우기 모습 (Landsat)', False) # 처음에는 끄고 시작
m_baseline.addLayer(water_mask_baseline, {'palette': 'blue'}, '기준선: 계산된 우기 수역 (Our Result)')
m_baseline.addLayer(jrc_permanent_water, {'palette': 'red'}, '기준선: 영구 수역 (JRC Validation)')

# JRC seasonality 밴드 시각화
jrc_seasonality_vis = {
    'min': 1, # 1개월 이상 물에 잠기는 지역
    'max': 12, # 12개월 내내 물에 잠기는 지역 (영구 수역)
    'palette': [
        '#0000ff', # 12개월 (진한 파랑: 영구 수역)
        '#2395a7', # 10-11개월 (청록색)
        '#52e338', # 7-9개월 (녹색: 장기간 침수)
        '#9be338', # 4-6개월 (연두색: 중기간 침수)
        '#dfff77', # 1-3개월 (노랑: 단기간 침수)
        '#ffff00'  # 0개월 (밝은 노랑: 가끔 물에 잠김)
    ]
}
m_baseline.addLayer(jrc_data.select('seasonality'), jrc_seasonality_vis, 'JRC 물 계절성 (Diversity)', False)

# 레이어 컨트롤 및 지도 표시
m_baseline.addLayerControl()
m_baseline

Phase1 해석:
1. 왜 톤레삽 호수는 영구 수역에 포함되지 않았을까?
JRC의 영구 수역 정의는 1년 내내, 38년간 항상 물이 차 있어야 함
그러나 톤레삽 호수는 건기(3~4월)가 되면 극단적으로 쪼그라들어 일부 지역은 바닥을 드러냄. 따라서 JRC 데이터가 톤레삽 호수의 대부분을 영구 수역으로 표시하지 않은 것은 오히려 JRC 데이터가 매우 정확하다는 증거, 반면 우리가 계산한 파란색 우기 수역은 거대하게 확장된 우기의 모습을 정확하게 포착

2. 왜 강 하류 메콩 델타에 영구 수역이 넓게 퍼져 있을까?
메콩 델타는 자연적인 지역이 아니라, 세계 최대의 쌀 생산지 중 하나로, 1년 내내 물을 가둬놓고 벼농사를 짓는 논 지역임.

3. 왜 그러면 메콩 델타의 우기 모습은 보이지 않는 거야?
우리가 사용한 MNDWI(수정된 정규화 수분 지수)는 위성 이미지에서 Open Water의 표면을 탐지함. 물은 녹색광(Green)은 반사하지만, 단파적외선(SWIR)은 흡수하는 성질이 있음. MNDWI는 이 원리를 이용하여 (Green - SWIR) / (Green + SWIR) 수식으로 물을 찾아냄. 그러나 메콩 델타의 논은 우기가 되면 물로 가득 차지만, 그 위에서는 벼가 빽빽하게 자람. 그래서 mndwi.gt(0) 기준 통과 못함

# Phase 2: SAR(Sentinel-1) 기반 홍수(Flood) 분석
* 목표: 댐 건설 이후(2015-2024), 우기의 홍수 패턴 변화를 다중 편파 분석을 통해 깊이 있게 들여다 봄
* step1: 매년 우기(8-9월)의 홍수 면적을 VV와 VH 편파로 시계열 데이터 구축
* step2: Phase1에서 계산한 baseline_wet_area를 그래프와 함께 표시하여 과거와의 변화 비교
* step3: 2018년 사례로, VV와 VH 탐지한 홍수 범위 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 분석할 연도 범위 설정
post_dam_start_year = 2015
post_dam_end_year = 2024

# 물을 탐지하기 위한 SAR 후방산란계수(backscatter) 임계값(dB)
# 각 편파에 맞는 임계값 설정
water_threshold_vv = -16
water_threshold_vh = -22

# 연도별 홍수 면적(VV, VH)을 모두 저장할 리스트
flood_areas_comparison = []

print(f"VV와 VH 편파를 동시 분석하여 연도별 홍수 면적 계산을 시작합니다...")

for year in range(post_dam_start_year, post_dam_end_year + 1):
    start_date = f'{year}-08-01'
    end_date = f'{year}-09-30'

    # 공통 Sentinel-1 컬렉션 불러오기(편파 필터링 전)
    s1_collection_base = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.eq('instrumentMode', 'IW'))

    # 면적 계산을 위한 함수 정의
    def calculate_flood_area(polarization, threshold):
        s1_filtered = s1_collection_base \
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
            .select(polarization)

        # 컬렉션에 이미지가 없는 경우 0을 반환하여 에러 방지
        image_count = s1_filtered.size().getInfo()
        if image_count == 0:
            return 0

        s1_min = s1_filtered.min().clip(aoi)
        water_map = s1_min.lt(threshold).selfMask()
        area = water_map.multiply(ee.Image.pixelArea()) \
            .reduceRegion(reducer=ee.Reducer.sum(), geometry=aoi, scale=30, maxPixels=1e12)

        return ee.Number(area.get(polarization)).divide(1e6).getInfo()

    # 각 편파별 면적 계산
    area_km2_vv = calculate_flood_area('VV', water_threshold_vv)
    area_km2_vh = calculate_flood_area('VH', water_threshold_vh)

    # 결과 저장
    flood_areas_comparison.append({
        'year': year,
        'area_km2_vv': area_km2_vv,
        'area_km2_vh': area_km2_vh
    })
    print(f" - {year}년: VV 면적={area_km2_vv:,.2f} km², VH 면적={area_km2_vh:,.2f} km²")

print("계산이 완료되었습니다.")

# 결과를 하나의 DataFrame으로 변환
df_comparison = pd.DataFrame(flood_areas_comparison)

# 비교 그래프 시각화
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 6))

# VV와 VH 면적을 꺾은선 그래프로 플롯
ax.plot(df_comparison['year'], df_comparison['area_km2_vv'], marker='o', linestyle='-', color='skyblue', label='Flood Area (VV Pol)')
ax.plot(df_comparison['year'], df_comparison['area_km2_vh'], marker='s', linestyle='--', color='royalblue', label='Flood Area (VH Pol)')

# 과거 평균 우기 면적 기준선을 빨간 점선으로 추가
ax.axhline(y=baseline_wet_area_km2, color='red', linestyle='--', linewidth=2, label=f'Pre-Dam Avg. (2005-08): {int(baseline_wet_area_km2):,} km²')

# 그래프 제목 및 축 레이블 설정
ax.set_title('Mekong Annual Flood Extent vs. Pre-Dam Baseline', fontsize=16)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Flood Area (km²)', fontsize=12)
ax.tick_params(axis='x', rotation=45)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend() # 범례 표시
ax.grid(True)

plt.tight_layout()
plt.show()

# 결과 확인
print("\n[VV vs VH 편파 기반 홍수 면적 비교 분석 결과]")
print(df_comparison)

Phase2 결과:
1. 다중 편파 분석 결과, 모든 연도에서 VH편파로 탐지된 면적이 VV편파 보다 넓게 나타남: 이는 Open Water뿐만 아니라, 광범위한 농경지 및 식생 지역까지 포함함을 의미

2. 홍수 면적은 2018년에 최대치(VH기준 79,259)를 기록한 후 급감했다가, 과거 평균보다 높은 수준에서 변동하는 새로운 패턴

3. 2018-2019년 구간의 비대칭적 변화: 면적 감소폭이 두 편파에서 다르게 나타남 VV면적(열린 물)은 전년 대비 약 27% 감소한 반면, VH면적은 약 17% 감소에 그쳐, 두 값의 상대적 격차가 벌어지는 현상 관측

In [ ]:
# VV vs VH 홍수 범위 비교 지도(2018년 예시)
year_to_visualize = 2018
start_date = f'{year_to_visualize}-08-01'
end_date = f'{year_to_visualize}-09-30'
water_threshold_vv = -16
water_threshold_vh = -22

# 각 편파별 홍수 지도(Water Map) 생성
def create_water_map(polarization, threshold):
    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', polarization)) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select(polarization)
    s1_min = s1_collection.min().clip(aoi)
    return s1_min.lt(threshold).selfMask()

water_map_vv = create_water_map('VV', water_threshold_vv)
water_map_vh = create_water_map('VH', water_threshold_vh)

# 지도 생성 및 시각화
m_compare = geemap.Map(center=[11.5, 105.5], zoom=8)
legend_dict = {
    'Open Water Flood (Detected by VV & VH)': 'add8e6', # 하늘색
    'Vegetated Area Flood (Detected by VH only)': '00008b' # 진한 파랑
}
m_compare.add_legend(title=f'{year_to_visualize} Flood Type', legend_dict=legend_dict)

# VV로 탐지한 홍수(Open Water)를 하늘색으로 표시
m_compare.addLayer(water_map_vv, {'palette': 'add8e6'}, 'Flood Area (VV - Open Water)')

# VH로만 탐지된 추가적인 홍수(식생 지역)를 진한 파란색으로 표시
vh_only_flood = water_map_vh.subtract(water_map_vv.unmask(0)).selfMask()

# vh.subtract(vv)는 두 영역의 차집합을 계산
m_compare.addLayer(vh_only_flood, {'palette': '00008b'}, 'Additional Flood Area (VH only)')
m_compare.addLayerControl()
m_compare

# Phase 3: SAR(Sentinel-1) 기반 가뭄(Drought) 분석
* 건기의 물 부족 현상을 분석하여, 댐 건설 이후 가뭄 변화를 보여줌
* 매년 건기(3-4월)의 최소 수역 면적을 VH 편파로 계산하여 시계열 구축

In [ ]:
# 과거 평균 건기 수역 면적 기준선 계산
print("과거(2005-2008) 평균 건기 수역 면적 기준선을 계산합니다...")

# MNDWI 함수 정의
def calculate_mndwi(image):
    return image.normalizedDifference(['SR_B2', 'SR_B5']).rename('MNDWI')

# 건기 기준선의 수역 마스크 생성 및 면적 계산
mndwi_dry_baseline = calculate_mndwi(landsat_dry_baseline)
water_mask_dry_baseline = mndwi_dry_baseline.gt(0).selfMask()
baseline_dry_area = water_mask_dry_baseline.multiply(ee.Image.pixelArea()) \
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30,
        maxPixels=1e12
    )

# 최종 기준선 값 (km²)
baseline_dry_area_km2 = ee.Number(baseline_dry_area.get('MNDWI')).divide(1e6).getInfo()
print(f"-> 계산 완료! 과거 평균 건기 수역 면적: {baseline_dry_area_km2:,.2f} km²")

In [ ]:
# SAR(VH 편파) 기반 연도별 건기 수역 면적 계산
analysis_start_year = 2015
analysis_end_year = 2024
water_threshold_vh = -22 # VH 편파 임계값

dry_season_areas = []
print(f"\nVH 편파를 사용하여 연도별 건기 수역 면적(가뭄) 분석을 시작합니다...")

for year in range(analysis_start_year, analysis_end_year + 1):
    start_date = f'{year}-03-01'
    end_date = f'{year}-04-30'

    s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi).filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')).select('VH')

    def calculate_surface_water_area(s1_coll, polarization, threshold):
        image_count = s1_coll.size().getInfo()
        if image_count == 0: return 0
        s1_min = s1_coll.min().clip(aoi)
        water_map = s1_min.lt(threshold).selfMask()
        area = water_map.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(), geometry=aoi, scale=30, maxPixels=1e12)
        return ee.Number(area.get(polarization)).divide(1e6).getInfo()

    area_km2_dry = calculate_surface_water_area(s1_collection, 'VH', water_threshold_vh)
    dry_season_areas.append({'year': year, 'area_km2_dry': area_km2_dry})
    print(f" - {year}년 건기 수역 면적: {area_km2_dry:,.2f} km²")

print("계산이 완료되었습니다.")
df_drought = pd.DataFrame(dry_season_areas)

In [ ]:
# 가뭄 분석 그래프 시각화
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(12, 6))

bars = ax.bar(df_drought['year'], df_drought['area_km2_dry'], color='skyblue', label='Dry Season Water Area')
ax.axhline(y=baseline_dry_area_km2, color='red', linestyle='--', linewidth=2, label=f'Pre-Dam Avg. (2005-08): {int(baseline_dry_area_km2):,} km²')

# 기준선보다 낮은 바(가뭄이 심한 해)를 다른 색으로 강조
for bar in bars:
    if bar.get_height() < baseline_dry_area_km2:
        bar.set_color('salmon')

ax.set_title('Mekong Dry Season Water Extent (Drought Analysis)', fontsize=16)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Surface Water Area (km²)', fontsize=12)
ax.tick_params(axis='x', rotation=45)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.show()

# 결과 확인
print("\n[가뭄 분석 결과 요약]")
print(df_drought)

# Phase 4: 종합 분석 및 최종 결론
* 홍수(우기 최대 면적)와 가뭄(건기 최소 면적) 데이터를 연간 수자원 변동성 파악
* 홍수 면적과 강수량(CHIRPS 데이터) 비교하여, 관측된 변화가 자연 현상만으로 설명될 수 없음 증명

In [ ]:
# 홍수와 가뭄 데이터를 'year' 기준으로 하나의 DataFrame으로 합치기
df_comprehensive = pd.merge(df_comparison, df_drought, on='year')

# 이중 축 그래프 시각화
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(14, 7))

# 왼쪽 Y축: 홍수(우기) 면적
color_wet = 'royalblue'
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Wet Season Max Area (Flood, km²)', color=color_wet, fontsize=12)
ax1.plot(df_comprehensive['year'], df_comprehensive['area_km2_vh'], color=color_wet, marker='s', linestyle='--', label='Flood Extent (VH)')
ax1.tick_params(axis='y', labelcolor=color_wet)
ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax1.axhline(y=baseline_wet_area_km2, color=color_wet, linestyle='--', alpha=0.5, linewidth=2, label=f'Pre-Dam Wet Avg.')

# 오른쪽 Y축: 가뭄(건기) 면적
ax2 = ax1.twinx()
color_dry = 'salmon'
ax2.set_ylabel('Dry Season Min Area (Drought, km²)', color=color_dry, fontsize=12)
ax2.plot(df_comprehensive['year'], df_comprehensive['area_km2_dry'], color=color_dry, marker='o', linestyle='-', label='Drought Extent (VH)')
ax2.tick_params(axis='y', labelcolor=color_dry)
ax2.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax2.axhline(y=baseline_dry_area_km2, color=color_dry, linestyle='--', alpha=0.5, linewidth=2, label=f'Pre-Dam Dry Avg.')

# 그래프 제목 및 레이아웃
plt.title('Mekong Water System Volatility (Post-Dam Era)', fontsize=18, pad=20)
ax1.set_xticks(df_comprehensive['year'])
ax1.tick_params(axis='x', rotation=45)
fig.tight_layout()
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')
plt.show()

In [ ]:
# 연도별 강수량 데이터 계산
print("연도별 우기(8-9월) 총 강수량 계산을 시작합니다...")

precipitation_data = []
for year in range(post_dam_start_year, post_dam_end_year + 1):
    start_date = f'{year}-08-01'
    end_date = f'{year}-09-30'

    # CHIRPS Daily 강수량 데이터셋 로드
    chirps_collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
        .filterDate(start_date, end_date) \
        .filterBounds(aoi)

    # 해당 기간의 총 강수량 계산
    total_precipitation_image = chirps_collection.sum().clip(aoi)

    # AOI 내의 평균 총 강수량 계산 (단위: mm)
    precip_stats = total_precipitation_image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=5566,  # CHIRPS 데이터의 해상도
        maxPixels=1e12
    )

    # 결과가 유효한지 확인 후 리스트에 추가
    precip_mm = precip_stats.get('precipitation').getInfo()
    if precip_mm is not None:
        precipitation_data.append({
            'year': year,
            'precipitation_mm': precip_mm
        })
        print(f" - {year}년 평균 총 강수량: {precip_mm:,.2f} mm")
    else:
        print(f" - {year}년 데이터 없음.")

# 강수량 데이터를 DataFrame으로 변환
df_precipitation = pd.DataFrame(precipitation_data)

# 기존 데이터와 강수량 데이터를 병합하여 최종 분석용 DataFrame 생성
df_final_analysis = pd.merge(df_comprehensive, df_precipitation, on='year')

print("\n[최종 분석 데이터셋]")
print(df_final_analysis)

# 이중 축 그래프 시각화(홍수 면적 vs. 강수량)
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(14, 7))

# ax1(왼쪽 Y축)에 '홍수 면적'을 막대 그래프로 그림
color_flood = 'skyblue'
ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Flood Area (km²)', color=color_flood, fontsize=12)
ax1.bar(df_final_analysis['year'], df_final_analysis['area_km2_vh'], color=color_flood, label='Flood Area (VH)')
ax1.tick_params(axis='y', labelcolor=color_flood)
ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
ax1.axhline(y=baseline_wet_area_km2, color='firebrick', linestyle='--', linewidth=2, label=f'Pre-Dam Wet Avg.')

# ax2(오른쪽 Y축)에 '총 강수량'을 꺾은선 그래프로 그림
ax2 = ax1.twinx()
color_precip = 'royalblue'
ax2.set_ylabel('Total Precipitation (mm)', color=color_precip, fontsize=12)
ax2.plot(df_final_analysis['year'], df_final_analysis['precipitation_mm'], color=color_precip, marker='o', linestyle='-', linewidth=2, label='Precipitation (mm)')
ax2.tick_params(axis='y', labelcolor=color_precip)

# 두 축의 범례를 하나로 통합하여 표시
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

# 그래프 제목 및 레이아웃 설정
plt.title('Mekong Flood Area vs. Precipitation (Post-Dam Era)', fontsize=18, pad=20)
ax1.set_xticks(df_final_analysis['year'])
ax1.tick_params(axis='x', rotation=45)
fig.tight_layout()
plt.show()